In [2]:
import sys,os,json
from mom_trans.momentum_transformer import TftDeepMomentumNetworkModel 
from settings.default import QUANDL_TICKERS
from mom_trans.model_inputs import ModelFeatures
import pandas as pd

In [3]:
experiment_name = "results/experiment_quandl_100assets_tft_cp12621_len252_notime_div_v1/2016-2023"
hp_directory = os.path.join(experiment_name + "/best_hyperparameters.json")
checkpoint_path = "results/experiment_quandl_100assets_tft_cp12621_len252_notime_div_v1/2016-2023/best/checkpoints"
params = {}
hiperparameters={}
with open('results/experiment_quandl_100assets_tft_cp12621_len252_notime_div_v1/2016-2023/fixed_params.json') as json_file:
    params = json.load(json_file)
with open('results/experiment_quandl_100assets_tft_cp12621_len252_notime_div_v1/2016-2023/best_hyperparameters.json') as json_file:
    hiperparameters = json.load(json_file)

features_file_path = "data/quandl_cpd_126lbw.csv"

In [4]:


raw_data = pd.read_csv(features_file_path, index_col=0, parse_dates=True)
raw_data["date"] = raw_data["date"].astype("datetime64[ns]")
train_interval = [2001, 2016, 2023]
changepoint_lbws = [126, 21]
ASSET_CLASS_MAPPING = dict(zip(QUANDL_TICKERS, ["COMB"] * len(QUANDL_TICKERS)))

model_features = ModelFeatures(
    raw_data,
    params["total_time_steps"],
    start_boundary=train_interval[0],
    test_boundary=train_interval[1],
    test_end=train_interval[2],
    changepoint_lbws=changepoint_lbws,
    split_tickers_individually=params["split_tickers_individually"],
    train_valid_ratio=params["train_valid_ratio"],
    add_ticker_as_static=(params["architecture"] == "TFT"),
    time_features=params["time_features"],
    lags=params["force_output_sharpe_length"],
    asset_class_dictionary=ASSET_CLASS_MAPPING,
    )

Index(['date', 'ticker', 'close', 'srs', 'daily_returns', 'daily_vol',
       'target_returns', 'norm_daily_return', 'norm_monthly_return',
       'norm_quarterly_return', 'norm_biannual_return', 'norm_annual_return',
       'macd_8_24', 'macd_16_48', 'macd_32_96', 'day_of_week', 'day_of_month',
       'week_of_year', 'month_of_year', 'year', 'cp_rl_126', 'cp_score_126',
       'cp_rl_21', 'cp_score_21', 'sd_parkinson_126', 'skew_126',
       'bbands_mavg_126', 'close_above_sma22', 'ema_above_sma200',
       'close_above_vwap_20', 'close_above_vwap_50', 'lm_std2_126',
       'sharpe_126', 'q1_close_divergence_100', 'q50_close_divergence_100',
       'q75_close_divergence_50', 'rolling_mode_200', 'rolling_mode_500',
       'backcusum_rejections_10_126', 'exuber_126_2_sadf',
       'exuber_126_2_bsadf', 'autoarima_1_126_PointForecast',
       'autoarima_1_126_Lo80', 'autoarima_1_126_Hi95',
       'autoarima_last_126_PointForecast', 'autoarima_last_126_Lo80',
       'autoarima_sd_126_Lo95

In [5]:
    
model = TftDeepMomentumNetworkModel(
        experiment_name,
        experiment_name  + "novo",
        #hp_minibatch_size,
        **params,
        #**model_features.input_params,
        **{
            "column_definition": model_features.get_column_definition(),
            "num_encoder_steps": 0,  # TODO artefact
            "stack_size": 1,
            "num_heads": 4,  # TODO to fixed params
        },
    )

Deep Momentum Network params:
architecture = TFT
total_time_steps = 252
early_stopping_patience = 100
multiprocessing_workers = 32
num_epochs = 300
fill_blank_dates = False
split_tickers_individually = True
random_search_iterations = 10
evaluate_diversified_val_sharpe = True
train_valid_ratio = 0.9
time_features = False
force_output_sharpe_length = None
input_size = 36
output_size = 1
category_counts = [16]
static_input_loc = [35]
known_regular_inputs = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]
known_categorical_inputs = [0]
changepoint_lbws = [126, 21]
features_file_path = data/quandl_cpd_126lbw.csv
column_definition = [('ticker', <DataTypes.CATEGORICAL: 1>, <InputTypes.ID: 4>), ('date', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>), ('norm_daily_return', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('norm_monthly_return', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('norm_

2022-07-15 13:05:42.398612: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/anaconda3/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [42]:
a = model.load_model(hiperparameters)

In [43]:
m = a.load_weights("results/experiment_quandl_100assets_TEST_tft_cp21_len126_notime_div_v1/2016-2023/hp/experiment_quandl_100assets_TEST_tft_cp21_len126_notime_div_v1/trial_3857f6b59ac7edd1938cabdd02136291/checkpoints/epoch_0/checkpoint")

ValueError: Shapes (25, 160) and (57, 160) are incompatible

In [34]:
model.get_positions(
        model_features.test_sliding,
        a,
        sliding_window=True,
        years_geq=train_interval[1],
        years_lt=train_interval[2],
    )

ValueError: in user code:

    File "/usr/local/anaconda3/lib/python3.8/site-packages/keras/engine/training.py", line 1801, in predict_function  *
        return step_function(self, iterator)
    File "/usr/local/anaconda3/lib/python3.8/site-packages/keras/engine/training.py", line 1790, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/anaconda3/lib/python3.8/site-packages/keras/engine/training.py", line 1783, in run_step  **
        outputs = model.predict_step(data)
    File "/usr/local/anaconda3/lib/python3.8/site-packages/keras/engine/training.py", line 1751, in predict_step
        return self(x, training=False)
    File "/usr/local/anaconda3/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/anaconda3/lib/python3.8/site-packages/keras/engine/input_spec.py", line 264, in assert_input_compatibility
        ra